In [1]:
# install necessary libraries
! pip uninstall sqlalchemy --yes
! pip install sqlalchemy==1.4.46
! pip install PyMySQL

Found existing installation: SQLAlchemy 1.4.46
Uninstalling SQLAlchemy-1.4.46:
  Successfully uninstalled SQLAlchemy-1.4.46
  Using cached SQLAlchemy-1.4.46-cp311-cp311-win_amd64.whl (1.6 MB)


In [ ]:
# Lab | SQL Queries 9

# Instructions
# In this lab we will find the customers who were active in consecutive months of May and June. 
# Follow the steps to complete the analysis.

In [2]:
# Importing the necessary libraries to set a connection between Python and SQL
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass() # insert your password below after executing this cell

········


In [3]:
# Create a Python connection with SQL database and retrieve the results of the last two queries 
#(also mentioned below) as dataframes:

# Making the connection with the database
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank' # database_location/database_name
engine = create_engine(connection_string)

In [13]:
# Check the number of rentals for each customer for May
df_customer_may_rentals = pd.read_sql_query('''
select c.customer_id, concat(c.first_name, ' ', c.last_name) as 'customer_name', 
        count(r.rental_id) as 'number_of_may_rentals'
from sakila.customer c
join sakila.rentals_may r using(customer_id)
group by c.customer_id, c.first_name, c.last_name
order by count(rental_id) desc
''', engine)

df_customer_may_rentals.head()

,customer_id,customer_name,number_of_may_rentals
0,197,SUE PETERS,8
1,506,LESLIE SEWARD,7
2,109,EDNA WEST,7
3,269,CASSANDRA WALTERS,6
4,239,MINNIE ROMERO,6


In [14]:
# Check the number of rentals for each customer for June
df_customer_june_rentals = pd.read_sql_query('''
select c.customer_id, concat(c.first_name, ' ', c.last_name) as 'customer_name', 
        count(r.rental_id) as 'number_of_june_rentals'
from sakila.customer c
join sakila.rentals_june r using(customer_id)
group by c.customer_id, c.first_name, c.last_name
order by count(rental_id) desc
''', engine)

df_customer_june_rentals.head()


,customer_id,customer_name,number_of_june_rentals
0,31,BRENDA WRIGHT,11
1,454,ALEX GRESHAM,10
2,329,FRANK WAGGONER,9
3,295,DAISY BATES,9
4,561,IAN STILL,9


In [15]:
# Write a function that checks if customer borrowed more or less films in the month of June 
# as compared to May.
df_merged = df_customer_june_rentals.merge(df_customer_may_rentals, how='outer').sort_values(by='customer_name')
df_merged

,customer_id,customer_name,number_of_june_rentals,number_of_may_rentals
355,375,AARON SELBY,3.0,4.0
567,367,ADAM GOOCH,1.0,2.0
540,525,ADRIAN CLARY,1.0,1.0
343,217,AGNES BISHOP,3.0,2.0
384,389,ALAN KAHN,3.0,1.0
...,...,...,...,...
226,359,WILLIE MARKHAM,4.0,3.0
234,212,WILMA RICHARDS,4.0,NaN
133,190,YOLANDA WEAVER,5.0,2.0
261,174,YVONNE WATKINS,4.0,2.0


In [21]:
# Fucntion that checks if the customer borrowed more or less films in month June
df_merged['comparison_result'] = df_merged.apply(lambda row: 'less' if row['number_of_june_rentals'] < row['number_of_may_rentals']
                                                 else 'more' if row['number_of_june_rentals'] >  row['number_of_may_rentals']
                                                 else 'equal', axis=1)
df_merged.sort_values(by='customer_name')
    

,customer_id,customer_name,number_of_june_rentals,number_of_may_rentals,comparison_result
355,375,AARON SELBY,3.0,4.0,less
567,367,ADAM GOOCH,1.0,2.0,less
540,525,ADRIAN CLARY,1.0,1.0,equal
343,217,AGNES BISHOP,3.0,2.0,more
384,389,ALAN KAHN,3.0,1.0,more
...,...,...,...,...,...
226,359,WILLIE MARKHAM,4.0,3.0,more
234,212,WILMA RICHARDS,4.0,NaN,equal
133,190,YOLANDA WEAVER,5.0,2.0,more
261,174,YVONNE WATKINS,4.0,2.0,more
